In [102]:
import torch

import pandas as pd
import numpy as np
import seaborn as sns

import torch.nn as nn
import torch.optim as optim

https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/

**Все остальные фичи тоже можно юзать в лстм - модель так будет вообще лютой**
- пока делаю: X - это последовательность только close, y - последовательность close
- потом: X - это последовательность (close, max, min, volume, еще какая-то статистика для этого момента), y - последовательность close

In [103]:
path = "../data/btc-4h.csv"

df = pd.read_csv(
    path
)

df = df[["time", "close"]]
# df['time'] = pd.to_datetime(df['time'], unit='s')

df = df[["close"]].values.astype('float32')    # пока для начала - предсказываем последовательность 
                    # close, основываясь на последовательности close

In [104]:
train_size = int(0.7 * len(df))

test_size = len(df) - train_size
train, test = df[:train_size], df[train_size:]

**Фича для одного объекта из датасета - это последовательность из w штук в предыдущие w моменты времени цен**

In [106]:
def create_dataset(dataset, lookback):
    """
    Transform a time series into a prediction dataset. We predict lookback steps forward based on lookback steps before.
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i:i + lookback]
        target = dataset[i + 1:i + lookback + 1]
        X.append(feature)
        y.append(target)    
    # return torch.tensor(np.vstack(X)), torch.tensor(np.vstack(y))
    return torch.tensor(X), torch.tensor(y)

In [110]:
lookback = 5

X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([3061, 5, 1]) torch.Size([3061, 5, 1])
torch.Size([1309, 5, 1]) torch.Size([1309, 5, 1])


In [111]:
X_train

tensor([[[48828.1719],
         [48944.2305],
         [48501.8711],
         [49180.9805],
         [48978.6797]],

        [[48944.2305],
         [48501.8711],
         [49180.9805],
         [48978.6797],
         [48821.8711]],

        [[48501.8711],
         [49180.9805],
         [48978.6797],
         [48821.8711],
         [48790.0000]],

        ...,

        [[18885.3496],
         [18846.6191],
         [18876.3691],
         [18824.2500],
         [18920.9492]],

        [[18846.6191],
         [18876.3691],
         [18824.2500],
         [18920.9492],
         [19261.1895]],

        [[18876.3691],
         [18824.2500],
         [18920.9492],
         [19261.1895],
         [19362.0098]]])

In [108]:
class SimpleModel(nn.Module):
    def __init__(self):
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x